In [1]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 
import statistics
import datetime
from dateutil.relativedelta import relativedelta
import random
random.seed(10)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Cell to change
sp = 'sp500'

In [3]:
# List of dowloaded articles
dir_path = "data/" + sp + "/text/"
tickers = os.listdir(dir_path)
#tickers.remove('.DS_Store') # for mac
print(len(tickers))

411


In [4]:
# Import earning calls

texts = []
list_ids = []                # list of the call ids in the same order as the list "texts"
list_tickers = []            # list of the tickers in the same order as the list "texts"
number_texts_per_stock = []

for ticker in tickers:
    dir_path = "data/"+sp+"/text/"+ticker+"/" 
    list_ids1 = os.listdir(dir_path)
    ticker1 = os.path.basename(os.path.dirname(dir_path))
    ticker1 = ticker1.split()
    texts1 = []
    for s in list_ids1:
        with open(dir_path + s, encoding = 'ISO-8859-1') as f:
            t = f.read()
            texts1.append(t)
    texts = texts + texts1
    list_ids = list_ids + list_ids1 
    list_tickers = list_tickers + ticker1*len(list_ids1)
    number_texts_per_stock.append(len(list_ids1))

In [5]:
# Get dates
data_path = "data/"+sp+"/ticker/id_to_date" 
with open(data_path,"rb") as file :
    id_to_date = pickle.load(file)

In [6]:
# Create dataframe with earning calls informations

# Earning call id
df1 = pd.DataFrame(data={'id': list_ids})

# Date
df1['date']=list_ids
df1=df1.replace({'date': id_to_date})
display(df1) #len : 21385 for sp500 & 19341 for SP600

# Ticker
df1['ticker']= list_tickers

# Text
df1['text']= texts

# Remove the non-identified dates
df1=df1[df1['date']!=df1['id']]

# Quarter earning call is reporting (1 quarter before the publication date)
df1['date'] = pd.to_datetime(df1['date'])
df1['lagged_date']=df1.date - pd.offsets.DateOffset(months=3)
df1['earning_call_quarter'] = df1['lagged_date'].dt.to_period('Q')
df1 = df1.drop(columns=['lagged_date'])

# Sort by ticker and date
df1= df1.sort_values(['ticker','date'], ascending=True)
display(df1)

,id,date
0,4333164,2020-03-19
1,259823,2011-03-23
2,126709,2009-03-18
3,4188578,2018-07-19
4,2767535,2014-12-18
...,...,...
21380,4464743,2021-11-02
21381,4119927,2017-11-02
21382,4424373,2021-05-05
21383,1983531,2014-01-30


,id,date,ticker,text,earning_call_quarter
10464,15491,2006-08-14,A,Question-and-Answer Session\n \nOperator\n \n(...,2006Q2
10468,20551,2006-11-14,A,Question-and-Answer Session\n \nOperator\n \nT...,2006Q3
10484,27245,2007-02-15,A,Question-and-Answer Session\n \nOperator\n \n(...,2006Q4
10462,35499,2007-05-14,A,Question-and-Answer Session\n \nOperator\n \n(...,2007Q1
10506,44508,2007-08-14,A,Question-and-Answer Session\n \nOperator\n \nY...,2007Q2
...,...,...,...,...,...
20332,4426158,2021-05-08,XRAY,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
20340,4445512,2021-08-05,XRAY,Question-and-Answer Session\n \nOperator\n \n[...,2021Q2
20345,4465541,2021-11-04,XRAY,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
20300,4491389,2022-02-28,XRAY,Question-and-Answer Session\n \nOperator\n \n[...,2021Q4


In [7]:
# Keep datas from 2019Q1 to last earning call (2022Q1)
start_date = '2019Q1'
end_date = '2022Q1'
mask = (df1['earning_call_quarter'] >= start_date)
df2 = df1.loc[mask]
display(df2)

# Compute n_quarters : Number of quarters between start_date and end_date
t = pd.to_datetime(end_date).to_period(freq='Q')-pd.to_datetime(start_date).to_period(freq='Q')
n_quarters = n_quarters = t.n+1 # n_quarters = 13 in our case

,id,date,ticker,text,earning_call_quarter
10470,4264085,2019-05-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
10494,4285738,2019-08-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
10495,4309111,2019-11-26,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
10499,4325101,2020-02-18,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
10469,4349475,2020-05-21,A,Question-and-Answer Session\n \nOperator\n \n[...,2020Q1
...,...,...,...,...,...
20332,4426158,2021-05-08,XRAY,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
20340,4445512,2021-08-05,XRAY,Question-and-Answer Session\n \nOperator\n \n[...,2021Q2
20345,4465541,2021-11-04,XRAY,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
20300,4491389,2022-02-28,XRAY,Question-and-Answer Session\n \nOperator\n \n[...,2021Q4


In [8]:
# Count number of texts per stock
texts_per_stock = df2[['id','ticker']].groupby(['ticker']).count().reset_index().rename(columns={'id': 'count'})
df_count = pd.DataFrame(texts_per_stock)

# Merge the initial DataFrame with the one containing the number of observation for each stock
df_merged = pd.merge(df2, df_count, on='ticker')
df_merged.reindex(columns=["id","date","earning_call_quarter","ticker","text","count"])

# Keep only the stocks that have n_quarters observations into the final DataFrame
df_final = df_merged[df_merged['count'] == n_quarters]
df_final = df_final.drop(columns = "count")

# Keep only the 100 first stocks
df_final = df_final[:1300]
display(df_final)

,id,date,ticker,text,earning_call_quarter
0,4264085,2019-05-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
1,4285738,2019-08-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
2,4309111,2019-11-26,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
3,4325101,2020-02-18,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
4,4349475,2020-05-21,A,Question-and-Answer Session\n \nOperator\n \n[...,2020Q1
...,...,...,...,...,...
1855,4422752,2021-04-30,DRE,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
1856,4442846,2021-07-29,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
1857,4463265,2021-10-28,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
1858,4482287,2022-01-27,DRE,Question-and-Answer Session \n \nOperator\n \n...,2021Q4


In [9]:
# Save ids in a csv file
df_final_ids = df_final['id']
display(df_final_ids)
df_final_ids.to_csv('df_100stocks_'+sp+'.csv')

0       4264085
1       4285738
2       4309111
3       4325101
4       4349475
         ...   
1855    4422752
1856    4442846
1857    4463265
1858    4482287
1859    4504759
Name: id, Length: 1300, dtype: object